# Food101 Classification with MobileNetV2
This notebook demonstrates how to train a MobileNetV2 model on a balanced subset of the Food101 dataset using TensorFlow and HuggingFace Datasets. It includes GPU support, data preprocessing, augmentation, and training with callbacks.

In [ ]:
# Install required library
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which

In [ ]:
import tensorflow as tf
from datasets import load_dataset
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

# --- 1. Check for GPU and enable memory growth ---
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    print("GPU доступний для використання.")
else:
    print("GPU не знайдено.")

GPU доступний для використання.


In [ ]:
from collections import Counter

# --- 2. Load dataset ---
dataset = load_dataset("food101")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

train-00000-of-00008.parquet:   0%|          | 0.00/490M [00:00<?, ?B/s]

train-00001-of-00008.parquet:   0%|          | 0.00/464M [00:00<?, ?B/s]

train-00002-of-00008.parquet:   0%|          | 0.00/472M [00:00<?, ?B/s]

...

validation-00000-of-00003.parquet:   0%|          | 0.00/423M [00:00<?, ?B/s]

validation-00001-of-00003.parquet:   0%|          | 0.00/413M [00:00<?, ?B/s]

validation-00002-of-00003.parquet:   0%|          | 0.00/426M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/75750 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/25250 [00:00<?, ? examples/s]

In [ ]:
# --- 3. Create a balanced middle subset for faster training ---
def balanced_middle_cut(ds, num_classes=101, samples_per_class=50):
    indices = []
    class_indices = {i: [] for i in range(num_classes)}
    for idx, label in enumerate(ds['label']):
        class_indices[label].append(idx)
    for label in range(num_classes):
        idxs = class_indices[label]
        if len(idxs) >= samples_per_class:
            start = (len(idxs) - samples_per_class) // 2
            indices.extend(idxs[start:start+samples_per_class])
        else:
            indices.extend(idxs)
    return ds.select(indices)

# Use a middle-sized balanced subset for training and validation
middle_train = balanced_middle_cut(dataset['train'], samples_per_class=50)
middle_val = balanced_middle_cut(dataset['validation'], samples_per_class=25)

In [ ]:
# --- 4. Preprocessing and augmentation functions ---
def preprocess_and_augment(example, img_size=(128, 128)):
    image = tf.image.resize(tf.convert_to_tensor(example['image']), img_size)
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, max_delta=0.2)
    image = tf.image.random_contrast(image, lower=0.8, upper=1.2)
    image = tf.clip_by_value(image, 0.0, 255.0) / 255.0
    label = tf.cast(example['label'], tf.int32)
    return {'image': image, 'label': label}

def preprocess_only(example, img_size=(128, 128)):
    image = tf.image.resize(tf.convert_to_tensor(example['image']), img_size) / 255.0
    label = tf.cast(example['label'], tf.int32)
    return {'image': image, 'label': label}

In [ ]:
# --- 5. Convert to tf.data.Dataset ---
train_data = middle_train.map(preprocess_and_augment).to_tf_dataset(
    columns=['image'], label_cols='label', shuffle=True, batch_size=64
).prefetch(tf.data.AUTOTUNE)

val_data = middle_val.map(preprocess_only).to_tf_dataset(
    columns=['image'], label_cols='label', shuffle=False, batch_size=64
).prefetch(tf.data.AUTOTUNE)

Map:   0%|          | 0/5050 [00:00<?, ? examples/s]

In [ ]:
import tensorflow as tf
from datasets import load_dataset
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

# --- 5. Використання стратегії для тренування на кількох GPU (якщо доступно) ---
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    # --- 6. Визначення моделі ---
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
    base_model.trainable = False  # Тільки верхній шар буде тренуватись

    x = GlobalAveragePooling2D()(base_model.output)
    x = Dropout(0.5)(x)
    output = Dense(101, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=output)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    model.summary()

    # --- 7. Callbacks ---
    callbacks = [
        ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1),
        EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    ]

    # --- 8. Тренування моделі ---
    history = model.fit(
        train_data,
        validation_data=val_data,
        epochs=20,
        callbacks=callbacks
    )


In [ ]:
# --- 9. Оцінка моделі ---
loss, accuracy = model.evaluate(val_data)
print(f"Validation Loss: {loss}")
print(f"Validation Accuracy: {accuracy}")

# --- 10. Збереження моделі ---
model.save('food101_model.h5')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

# --- 11. Візуалізація результатів ---
def plot_training_history(history):
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.show()
plot_training_history(history)